In [1]:
from couplets_utils import *
from keras.layers import *
from keras.models import Model
from keras import backend as K
import numpy as np
import tensorflow as tf

%load_ext autoreload
%autoreload 2

/Users/zhangshulin_work/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
N_A = 128

In [3]:
sample = load_sample_datasets(vocabs_size=2000, max_len=30, batch_size=16, sample_size=1000, n_a=N_A)
sample_gen = sample['sample_gen']
index2word = sample['index2word']
word2index = sample['word2index']
vocabs_size = sample['vocabs_size']
max_len = sample['max_len']

In [4]:
print('sample_gen length :', len(sample_gen))

sample_gen length : 63


In [5]:
def create_train_model(n_x, n_a, Tx, keep_prob=0.5):    
    input = Input(shape=(Tx, n_x), name='x0')
    a_0_0 = Input(shape=(n_a,), name='a_0_0')
    c_0_0 = Input(shape=(n_a,), name='c_0_0')
    a_1_0 = Input(shape=(n_a,), name='a_1_0')
    c_1_0 = Input(shape=(n_a,), name='c_1_0')
        
    a_0 = a_0_0
    c_0 = c_0_0
    a_1 = a_1_0
    c_1 = c_1_0
    
    lstm_cell_0 = LSTM(units=n_a, return_state=True, name='lstm_0')
    lstm_cell_1 = LSTM(units=n_a, return_state=True, name='lstm_1')
    dense_layer_2 = Dense(units=n_x, activation='softmax', name='softmax_2')
        
    outputs = []
        
    for i in range(Tx):
        x = Lambda(lambda j: j[:, i, :])(input)
        x = Reshape(target_shape=(1, -1))(x)
        a_0, x, c_0 = lstm_cell_0(x, initial_state=[a_0, c_0])
        x = Dropout(rate=1-keep_prob)(x)
        x = Reshape(target_shape=(1, -1))(x)
        a_1, x, c_1 = lstm_cell_1(x, initial_state=[a_1, c_1])
        x = Dropout(rate=1-keep_prob)(x)
        x = dense_layer_2(x)
        outputs.append(x)
                            
    model = Model(inputs=[input, a_0_0, c_0_0, a_1_0, c_1_0], outputs=outputs)
        
    return model

In [6]:
train_model = create_train_model(vocabs_size, N_A, max_len)

In [7]:
train_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
train_model.fit_generator(sample_gen, epochs=1)

Epoch 1/1
63/63 [==============================] - 36s 571ms/step - loss: 157.3878 - softmax_2_loss_1: 7.5608 - softmax_2_loss_2: 7.5134 - softmax_2_loss_3: 7.4060 - softmax_2_loss_4: 7.2196 - softmax_2_loss_5: 6.8515 - softmax_2_loss_6: 7.1226 - softmax_2_loss_7: 7.6629 - softmax_2_loss_8: 5.8958 - softmax_2_loss_9: 7.6804 - softmax_2_loss_10: 7.5493 - softmax_2_loss_11: 7.5136 - softmax_2_loss_12: 7.2818 - softmax_2_loss_13: 6.6289 - softmax_2_loss_14: 6.8460 - softmax_2_loss_15: 6.7875 - softmax_2_loss_16: 5.0992 - softmax_2_loss_17: 3.9335 - softmax_2_loss_18: 3.5774 - softmax_2_loss_19: 3.9296 - softmax_2_loss_20: 3.6571 - softmax_2_loss_21: 3.6758 - softmax_2_loss_22: 3.5773 - softmax_2_loss_23: 3.5363 - softmax_2_loss_24: 3.3556 - softmax_2_loss_25: 3.2804 - softmax_2_loss_26: 2.8353 - softmax_2_loss_27: 2.5079 - softmax_2_loss_28: 2.3868 - softmax_2_loss_29: 2.2817 - softmax_2_loss_30: 2.2337 - softmax_2_acc_1: 0.0000e+00 - softmax_2_acc_2: 0.0000e+00 - softmax_2_acc_3: 0.0000e

In [9]:
train_model.save_weights('./weights.h5')

In [10]:
def create_infer_model(n_x, n_a, Tx):
    input = Input(shape=(Tx, n_x), name='x0')
    a_0_0 = Input(shape=(n_a,), name='a_0_0')
    c_0_0 = Input(shape=(n_a,), name='c_0_0')
    a_1_0 = Input(shape=(n_a,), name='a_1_0')
    c_1_0 = Input(shape=(n_a,), name='c_1_0')
    
    x = Lambda(lambda x: x[:, 0, :])(input)
    a_0 = a_0_0
    c_0 = c_0_0
    a_1 = a_1_0
    c_1 = c_1_0

    lstm_cell_0 = LSTM(units=n_a, return_state=True, name='lstm_0')
    lstm_cell_1 = LSTM(units=n_a, return_state=True, name='lstm_1')
    dense_layer_2 = Dense(units=n_x, activation='softmax', name='softmax_2')
    
    def one_hot(x):
        x = K.argmax(x)
        x = tf.one_hot(x, n_x) 
        return x
    
    def select_x(x, i):
        return tf.cond(
            tf.equal(tf.reduce_sum(input[:, i+1, :]), 0),
            lambda : x,
            lambda : input[:, i+1, :]
        )
        
    def select_output(x, i):
        return tf.cond(
            tf.less(i + 1, Tx),
            lambda : select_x(x, i),
            lambda : x
        )
    
    outputs = []
    
    for i in range(Tx - 1):              
        x = Reshape(target_shape=(1, -1))(x)
        a_0, x, c_0 = lstm_cell_0(x, initial_state=[a_0, c_0])
        x = Reshape(target_shape=(1, -1))(x)
        a_1, x, c_1 = lstm_cell_1(x, initial_state=[a_1, c_1])
        x = dense_layer_2(x)
        x = Lambda(one_hot)(x)
        
        x = Lambda(lambda x: select_output(x, i))(x)
        outputs.append(x)
        
        
    model = Model(inputs=[input, a_0_0, c_0_0, a_1_0, c_1_0], outputs=outputs)
    
    return model

In [11]:
infer_model = create_infer_model(vocabs_size, N_A, max_len)
infer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
infer_model.load_weights('./weights.h5')

In [13]:
def write_couplets(begin_text, infer_model):
    x = convert_text_to_onehot(begin_text, vocabs_size, max_len, word2index)
    a_0 = np.zeros((1, N_A))
    c_0 = np.zeros((1, N_A))
    a_1 = np.zeros((1, N_A))
    c_1 = np.zeros((1, N_A))
    
    result = infer_model.predict([x, a_0, c_0, a_1, c_1])
    result_text = convert_predict_to_text(np.array(result), index2word)
    
    return result_text

In [14]:
text = '天天向上'
result_text = write_couplets(text, infer_model)
print(result_text)

天天向上                         
